In [ ]:
%pylab

In [ ]:
%matplotlib inline

In [ ]:
import pystan, pickle, popmachine, scipy, os
import pandas as pd

from pystan_cache.pystan_cache import caching_stan

In [ ]:
gp_multi = caching_stan.stan_model(file='stan-models/gp_multi.stan')
gp_multi_marginal = caching_stan.stan_model(file='stan-models/gp_multi_marginal.stan')

In [ ]:
machine = popmachine.Machine('sqlite:///../popmachine_local/.popmachine.db')

plates = [u'20161010_PQ_osmo', u'20150517 PQ 3', u'20161107_PQ_osmo_combo',
       u'20150715 PQ 8', u'20150702 PQ 6', u'20150607 PQ 4',
       u'20150630 PQ 5', u'20150704 PQ 7', u'20150717 PQ 9']

In [ ]:
for plate in plates:
    
    fname = 'ura3-0.333mMPQ-replicate-%s-samples.pkl'%plate
    
    if fname in os.listdir('samples'):
        print plate, 'already done!'
        continue
    
    ds = machine.search(plates=[plate], Strain='ura3', **{'mM PQ':[0.0, .333,333.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    ds.meta.loc[ds.meta['mM PQ']=='333.0', 'mM PQ'] = '0.333'
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    
    ymean, ystd = y.mean(), y.std()
    y = (y-y.mean())/y.std()
    x = (xraw-xraw.mean())/xraw.std()
    
    dm = np.zeros((y.shape[1], 2))
    dm[:,0] = 1
    dm[:,1] = design['mM PQ'] != labels[0].index('0.0')
    
    p = dm.shape[1]
    n = x.shape[0]

    train_data = {
        'N': n,
        'P':y.shape[1],
        'K':dm.shape[1],
        'L':2,
        'prior':[1,2],     
        'design': dm
    }

    train_data['y'] = y.T
    train_data['x'] = x[:,0]

    train_data['alpha_prior'] = [[1,1], [.1, 1]]
    train_data['length_scale_prior'] = [[.5, .5], [.5,.5]]
    train_data['marginal_alpha_prior'] = [.5, .1]
    train_data['marginal_lengthscale_prior'] = [8, 2.0]

    train_data['sigma_prior'] = [.1,1.5]

    tsamples = gp_multi_marginal.sampling(data=train_data, chains=4, iter=2000, control = {'adapt_delta': 0.8})
    
    summary = tsamples.summary()
    summary = pd.DataFrame(summary['summary'], columns=summary['summary_colnames'], index=summary['summary_rownames'])
    
    tsamp = tsamples.extract(permuted=True)
    pickle.dump(tsamp, open('samples/%s'%fname, 'wb'))
    
    print plate
    print summary.Rhat.describe()

In [ ]:
for plate in plates:
    
    fname = 'ura3-0.333mMPQ-replicate-null-%s-samples.pkl'%plate
    
    if fname in os.listdir('samples'):
        print plate, 'already done!'
        continue
    
    ds = machine.search(plates=[plate], Strain='ura3', **{'mM PQ':[0.0, .333,333.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    
    ymean, ystd = y.mean(), y.std()
    y = (y-y.mean())/y.std()
    x = (xraw-xraw.mean())/xraw.std()
    
    dm = np.zeros((y.shape[1], 2))
    dm[:,0] = 1
    dm[:,1] = design['mM PQ'] != labels[0].index('0.0')
    
    p = dm.shape[1]
    n = x.shape[0]

    train_data = {
        'N': n,
        'P':y.shape[1],
        'K':dm.shape[1],
        'L':2,
        'prior':[1,2],     
        'design': dm
    }

    train_data['y'] = y.T
    train_data['x'] = x[:,0]

    train_data['alpha_prior'] = [[1,1], [.1, 1]]
    train_data['length_scale_prior'] = [[.5, .5], [.5,.5]]
    train_data['marginal_alpha_prior'] = [.5, .1]
    train_data['marginal_lengthscale_prior'] = [8, 2.0]

    train_data['sigma_prior'] = [.1,1.5]

    tsamples = gp_multi.sampling(data=train_data, chains=4, iter=2000, control = {'adapt_delta': 0.8})
    
    summary = tsamples.summary()
    summary = pd.DataFrame(summary['summary'], columns=summary['summary_colnames'], index=summary['summary_rownames'])
    
    tsamp = tsamples.extract(permuted=True)
    pickle.dump(tsamp, open('samples/%s'%fname, 'wb'))
    
    print plate
    print summary.Rhat.describe()

In [ ]:
plt.figure(figsize=(8, 4))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.333mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.333mMPQ-replicate-null-%s-samples.pkl'%plate, 'rb'))

    plt.subplot(121)    
    temp = nsamp['f'][:,1,:31]
    temp = temp*ystd #+ ymean
    plt.plot(xraw, temp.mean(0),)
    plt.fill_between(xraw[:,0], 
                     temp.mean(0)-2*temp.std(0),
                     temp.mean(0)+2*temp.std(0),alpha=.1)

    plt.subplot(122)    
    temp = tsamp['f'][:,1,:31]
    temp = temp*ystd #+ ymean
    plt.plot(xraw, temp.mean(0),)
    plt.fill_between(xraw[:,0], 
                     temp.mean(0)-2*temp.std(0),
                     temp.mean(0)+2*temp.std(0),alpha=.1)
    

plt.subplot(121)
plt.xlabel('time (h)', fontsize=16)
plt.ylabel('log(OD)', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.plot([xraw.min(), xraw.max()], [0,0], c='k')

plt.subplot(122)
plt.xlabel('time (h)', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.plot([xraw.min(), xraw.max()], [0,0], c='k')

plt.tight_layout()

plt.savefig('figures/ura3_0.333mMPQ-replicate-effectFunctions.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(8, 4))

for p, plate in enumerate(plates):
    
    ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

    ds.log()
    ds.filter()
    ds.trim(5)
    ds.poly_scale(2, groupby=['plate', 'mM PQ'])

    ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.333mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.333mMPQ-replicate-null-%s-samples.pkl'%plate, 'rb'))
    
    temp = nsamp['f'][:,1,:31]
    temp = temp*ystd #+ ymean
    
    temp2 = tsamp['f'][:,1,:31]
    temp2 = temp2*ystd #+ ymean
    
    plt.boxplot([np.log10(temp2.var(0)/temp.var(0))], positions=[p], showfliers=False)

plt.xticks([])
plt.plot([-1, len(plates)], [0,0], c='k')   
plt.xlim(-1, len(plates))
plt.yticks(fontsize=16)

plt.savefig('figures/ura3_0.333mMPQ-replicate-effectFunctionInterval.pdf', bbox_inches='tight')

In [ ]:
plt.figure(figsize=(9,9))

for p, plate in enumerate(plates):
    
#     ds = machine.search(plates = [plate], Strain='ura3', **{'mM PQ':[0.0], 'M NaCl':[4.2, None]})

#     ds.log()
#     ds.filter()
#     ds.trim(5)
#     ds.poly_scale(2, groupby=['plate', 'mM PQ'])

#     ds.data = ds.data.iloc[::3,:]
    
    xraw ,y, design, labels = ds.build(['mM PQ', 'plate'],scale=True)
    ymean, ystd = y.mean(), y.std()
    
    xraw = xraw[:31,:]
    
    tsamp = pickle.load(open('samples/ura3-0.333mMPQ-replicate-%s-samples.pkl'%plate, 'rb'))
    nsamp = pickle.load(open('samples/ura3-0.333mMPQ-replicate-null-%s-samples.pkl'%plate, 'rb'))
    
    temp = tsamp['sigma'] + tsamp['marginal_alpha']
    temp2 = nsamp['sigma']

    plt.subplot(3, 3, p+1)
    plt.boxplot([temp2], positions=[0], showfliers=False, vert=False, widths=[.5])
    plt.boxplot([tsamp['sigma']], positions=[1], showfliers=False, boxprops={'color':'blue'}, widths=[.5], vert=False)
    plt.boxplot([tsamp['marginal_alpha']], positions=[2], showfliers=False, boxprops={'color':'red'}, widths=[.5], vert=False)
    plt.boxplot([tsamp['sigma'] + tsamp['marginal_alpha']], positions=[3], showfliers=False, boxprops={'color':'green'}, widths=[.5], vert=False)
    plt.ylim(-1, 4)
    
    if p % 3 == 0:
        plt.yticks(range(4), ['']*4)
    else:
        plt.yticks([])
        
    plt.xticks(fontsize=12)
    
plt.tight_layout()
plt.savefig('figures/ura3_0.333mMPQ-varianceTotal-replicate.pdf', bbox_inches='tight')